 # <center> Lab 4 : Twitter sentiment Analysis

### Name : DAHAMOU Abdelilah

## part 1 : practical example using SentiWordnet

#**Importing Required Libraries**

In [1]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

sentence = "One of the best movie of all time. Period."

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\sentiwordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


#**Text PreProcessing**

#### (1) Removing Punctuations

In [2]:
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

for x in sentence: 
  if x in punctuations: 
    sentence = sentence.replace(x, "")

print(sentence)

One of the best movie of all time Period


#### (2) Change Case + Tokenization

In [3]:
Tokens = nltk.word_tokenize(sentence.lower())
print(Tokens)

['one', 'of', 'the', 'best', 'movie', 'of', 'all', 'time', 'period']


#### (3) Removing Stop Words - a, an, the, are, is etc.

In [4]:
stop_words = set(stopwords.words('english'))
clean_Tokens = [word for word in Tokens if word not in stop_words]
print(clean_Tokens)

['one', 'best', 'movie', 'time', 'period']


#### (4) Lemmatization

In [5]:
lemma = [lemmatizer.lemmatize(word) for word in clean_Tokens]

print(lemma)

['one', 'best', 'movie', 'time', 'period']


#### (5) POS Tagging

A part-of-speech tagger, or POS-tagger, processes a sequence of words, and attaches a part of speech tag to each word (don't forget to import nltk)

In [6]:
lemma

['one', 'best', 'movie', 'time', 'period']

In [7]:
help(nltk.pos_tag)

Help on function pos_tag in module nltk.tag:

pos_tag(tokens, tagset=None, lang='eng')
    Use NLTK's currently recommended part of speech tagger to
    tag the given list of tokens.
    
        >>> from nltk.tag import pos_tag
        >>> from nltk.tokenize import word_tokenize
        >>> pos_tag(word_tokenize("John's big idea isn't all that bad."))
        [('John', 'NNP'), ("'s", 'POS'), ('big', 'JJ'), ('idea', 'NN'), ('is', 'VBZ'),
        ("n't", 'RB'), ('all', 'PDT'), ('that', 'DT'), ('bad', 'JJ'), ('.', '.')]
        >>> pos_tag(word_tokenize("John's big idea isn't all that bad."), tagset='universal')
        [('John', 'NOUN'), ("'s", 'PRT'), ('big', 'ADJ'), ('idea', 'NOUN'), ('is', 'VERB'),
        ("n't", 'ADV'), ('all', 'DET'), ('that', 'DET'), ('bad', 'ADJ'), ('.', '.')]
    
    NB. Use `pos_tag_sents()` for efficient tagging of more than one sentence.
    
    :param tokens: Sequence of tokens to be tagged
    :type tokens: list(str)
    :param tagset: the tagset to be u

In [8]:
pos_val = nltk.pos_tag(lemma)
print(pos_val)

pos=neg=obj=count=0

[('one', 'CD'), ('best', 'JJS'), ('movie', 'NN'), ('time', 'NN'), ('period', 'NN')]


CC, a coordinating conjunction;

RB, or adverbs;  

IN, a preposition; 

NN, a noun; 

JJ, an adjective.

The most popular tag set is Penn Treebank tagset. Most of the already trained taggers for English are trained on this tag set. To view the complete list, follow this link--> https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [11]:
# We can get more details about any POS tag using help funciton of NLTK as follows.
nltk.download('tagsets')
nltk.help.upenn_tagset("JJS$")

JJS: adjective, superlative
    calmest cheapest choicest classiest cleanest clearest closest commonest
    corniest costliest crassest creepiest crudest cutest darkest deadliest
    dearest deepest densest dinkiest ...


[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [13]:
nltk.download('universal_tagset')
pos_val2 = nltk.pos_tag(lemma, tagset='universal')
print(pos_val2)

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...


[('one', 'NUM'), ('best', 'ADJ'), ('movie', 'NOUN'), ('time', 'NOUN'), ('period', 'NOUN')]


[nltk_data]   Unzipping taggers\universal_tagset.zip.


#**Functions for Sentiment Scoring**

In [14]:
# Convert between the PennTreebank tags to simple Wordnet tags
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

you can find the whol peeTreebank here https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [15]:
# Returns list of pos-neg and objective score. But returns empty list if not present in senti wordnet.
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    #Lemmatization
    #lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    #if not lemma:
     #   return []

    #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
    #Synset instances are the groupings of synonymous words that express the same concept. 
    #Some of the words have only one Synset and some have several.
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

In [16]:
get_sentiment('one', 'NUM')

['one.n.01', 0.0, 0.0, 1.0]

In [17]:
senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
print(senti_val)

[[], ['best.a.01', 0.75, 0.0, 0.25], ['movie.n.01', 0.0, 0.0, 1.0], ['time.n.01', 0.0, 0.0, 1.0], ['time_period.n.01', 0.125, 0.125, 0.75]]


#**Aggregating Scores**

In [18]:
for i in range(len(senti_val)):
  try:
    pos = pos + senti_val[i][1]
    neg = neg + senti_val[i][2]
    obj = obj + senti_val[i][3]
  
  except:
    continue

In [19]:
print("Positive weight : {0} ".format(pos))
print("Negative weight : {0} ".format(neg))
print("Sentiment of the statement is {0} ".format(pos - neg))

Positive weight : 0.875 
Negative weight : 0.125 
Sentiment of the statement is 0.75 


## part 2: twitter sentiment Analysis

In [90]:
#data
#!pip install openpyxl

In [20]:
import pandas as pd
Tweet = pd.read_excel("US_Airline_Tweets.xlsx") # Weather_Final.csv # Airline_Final.csv

In [21]:
data=Tweet['Tweets']
data

0                    @VirginAmerica What @dhepburn said.
1      @VirginAmerica plus you've added commercials t...
2      @VirginAmerica I didn't today... Must mean I n...
3      @VirginAmerica it's really aggressive to blast...
4      @VirginAmerica and it's a really big bad thing...
                             ...                        
995    @united #UnitedAirlines Pls Fix #AspenBaggageF...
996    @united Read my bio. See who I work with. I ha...
997    @united Does customer care have email or a pho...
998    Thank you â€œ@united: @TRUU_Tall I can certain...
999    @united In the process of recovering their car...
Name: Tweets, Length: 1000, dtype: object

In [22]:
import re 
#preprocessing
def tweet_preprocess(raw_tweet):
    
    raw_tweet = re.sub("@\w+","",raw_tweet).strip()
    raw_tweet = re.sub("http\S+","",raw_tweet).strip()
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    lemma = [lemmatizer.lemmatize(word) for word in meaningful_words]
    return lemma#( " ".join( meaningful_words )) 

In [23]:
new_data=[]
for raw in data:
    raw=tweet_preprocess(raw)
    new_data.append(raw)
    print(raw)

['said']
['plus', 'added', 'commercial', 'experience', 'tacky']
['today', 'must', 'mean', 'need', 'take', 'another', 'trip']
['really', 'aggressive', 'blast', 'obnoxious', 'entertainment', 'guest', 'face', 'amp', 'little', 'recourse']
['really', 'big', 'bad', 'thing']
['seriously', 'would', 'pay', 'flight', 'seat', 'playing', 'really', 'bad', 'thing', 'flying', 'va']
['yes', 'nearly', 'every', 'time', 'fly', 'vx', 'ear', 'worm', 'go', 'away']
['really', 'missed', 'prime', 'opportunity', 'men', 'without', 'hat', 'parody']
['well']
['amazing', 'arrived', 'hour', 'early', 'good']
['know', 'suicide', 'second', 'leading', 'cause', 'death', 'among', 'teen']
['lt', 'pretty', 'graphic', 'much', 'better', 'minimal', 'iconography']
['great', 'deal', 'already', 'thinking', 'nd', 'trip', 'amp', 'even', 'gone', 'st', 'trip', 'yet', 'p']
['flying', 'fabulous', 'seductive', 'sky', 'u', 'take', 'stress', 'away', 'travel']
['thanks']
['sfo', 'pdx', 'schedule', 'still', 'mia']
['excited', 'first', 'cros

['something', 'wrong', 'website', 'safari', 'iphone']
['thanks', 'gate', 'checking', 'baggage', 'full', 'flight', 'dfw', 'lax', 'giving', 'early', 'boarding', 'sweet']
['already', 'checked', 'page', 'expired', 'x']
['boo', 'refunding', 'seat', 'upgrade', 'fee', 'want', 'buy', 'first', 'place']
['share', 'rise', 'q', 'financial', 'result', 'usa', 'today']
['add', 'pillow', 'instead', 'lie', 'flat', 'seat', 'first', 'class', 'arm', 'race']
['see']
['way', 'take', 'advantage', 'mayweatherpacquiao']
['know', 'flying', 'virgin', 'fight', 'mayweatherpacquiao']
['inflight', 'team', 'make', 'experience', 'amazing']
['cutest', 'salt', 'pepper', 'shaker', 'ever', 'think', 'guy', 'get', 'better']
['moved', 'seat', 'notice', 'better', 'seat', 'cabin', 'select', 'behind', 'row', 'selected', 'disappointed']
['share', 'q', 'result']
['fav', 'airline']
['hot', 'female', 'pilot', 'sweet', 'dca', 'sfo']
['guy', 'going', 'start', 'flying', 'paris', 'welcome']
['policy', 'flying', 'surgery', 'still', 'wai

['iah', 'charlotte', 'baggage', 'claim', 'rep', 'latrice', 'h', 'customerservice', 'non', 'existent', 'ignored', 'customer', 'inappropriately', 'touched', 'customer']
['kewl', 'also', 'let', 'know', 'overhead', 'bin', 'row', 'inop', 'love', 'good', 'abbreve']
['thanks', 'help', 'got', 'need', 'issue', 'guess', 'sit', 'awhile', 'longer']
['would', 'loved', 'respond', 'website', 'saw', 'really', 'long', 'form', 'business', 'new', 'seat', 'bad']
['refund', 'flight', 'voucher', 'another', 'flight', 'different', 'airline', 'expense', 'today', 'hour', 'drive', 'inconvenient']
['received', 'call', 'back', 'day', 'infant', 'car', 'seat', 'supposed', 'go', 'anywhere', 'child']
['thanks', 'concern', 'contact', 'customer', 'care', 'upon', 'return', 'australia']
['well', 'sitting', 'ground', 'time', 'waiting', 'gate', 'tiredofthis']
['need', 'guy', 'book', 'plane', 'concern', 'travel', 'someone', 'else', 'moving', 'forward']
['measly', 'e', 'certificate', 'appreciate', 'loyal', 'customer', 'wait',

['lot', 'report', 'system', 'failure', 'delaying', 'flight', 'last', 'week', 'currently', 'sitting', 'tarmac', 'ogg', 'hour']
['rebooting', 'chicago', 'dispatch', 'system', 'need', 'tired', 'sitting', 'plane']
['despite', 'shaky', 'connection', 'look', 'like', 'get', 'home', 'tonight', 'great', 'job', 'touch', 'go']
['however', 'completely', 'control', 'price', 'still', 'displayed']
['aircraft', 'closest', 'gate', 'flight', 'one', 'closest', 'gate', 'going', 'chicago']
['tell', 'file', 'claim', 'clothing', 'purchased']
['comically', 'return', 'flight', 'ase', 'two', 'united', 'flight', 'boarding', 'adjacent', 'gate', 'clear', 'plane', 'board']
['grk', 'file', 'reference']
['would', 'contact', 'discus', 'poor', 'experience', 'customer', 'service', 'would', 'give', 'shot']
['ua', 'pacific', 'rim', 'date', 'night', 'cut', 'constantly', 'randomly', 'one', 'spot', 'repeatably']
['trying', 'since', 'file', 'report']
['send', 'email', 'bad', 'experience', 'send', 'back', 'generic', 'response'

In [24]:
new_data

[['said'],
 ['plus', 'added', 'commercial', 'experience', 'tacky'],
 ['today', 'must', 'mean', 'need', 'take', 'another', 'trip'],
 ['really',
  'aggressive',
  'blast',
  'obnoxious',
  'entertainment',
  'guest',
  'face',
  'amp',
  'little',
  'recourse'],
 ['really', 'big', 'bad', 'thing'],
 ['seriously',
  'would',
  'pay',
  'flight',
  'seat',
  'playing',
  'really',
  'bad',
  'thing',
  'flying',
  'va'],
 ['yes', 'nearly', 'every', 'time', 'fly', 'vx', 'ear', 'worm', 'go', 'away'],
 ['really',
  'missed',
  'prime',
  'opportunity',
  'men',
  'without',
  'hat',
  'parody'],
 ['well'],
 ['amazing', 'arrived', 'hour', 'early', 'good'],
 ['know', 'suicide', 'second', 'leading', 'cause', 'death', 'among', 'teen'],
 ['lt', 'pretty', 'graphic', 'much', 'better', 'minimal', 'iconography'],
 ['great',
  'deal',
  'already',
  'thinking',
  'nd',
  'trip',
  'amp',
  'even',
  'gone',
  'st',
  'trip',
  'yet',
  'p'],
 ['flying',
  'fabulous',
  'seductive',
  'sky',
  'u',
  'ta

## To do : for each tweet get it sentiment using sentiwordnet
NB : add you name and ubmit your work under Lab4_submission 

In [44]:
L=[]
for i in range (0,len(new_data)):
    stop_words = set(stopwords.words('english'))
    clean_Tokens = [word for word in new_data[i] if word not in stop_words]
    L+=[clean_Tokens]
print(L)    

[['said'], ['plus', 'added', 'commercial', 'experience', 'tacky'], ['today', 'must', 'mean', 'need', 'take', 'another', 'trip'], ['really', 'aggressive', 'blast', 'obnoxious', 'entertainment', 'guest', 'face', 'amp', 'little', 'recourse'], ['really', 'big', 'bad', 'thing'], ['seriously', 'would', 'pay', 'flight', 'seat', 'playing', 'really', 'bad', 'thing', 'flying', 'va'], ['yes', 'nearly', 'every', 'time', 'fly', 'vx', 'ear', 'worm', 'go', 'away'], ['really', 'missed', 'prime', 'opportunity', 'men', 'without', 'hat', 'parody'], ['well'], ['amazing', 'arrived', 'hour', 'early', 'good'], ['know', 'suicide', 'second', 'leading', 'cause', 'death', 'among', 'teen'], ['lt', 'pretty', 'graphic', 'much', 'better', 'minimal', 'iconography'], ['great', 'deal', 'already', 'thinking', 'nd', 'trip', 'amp', 'even', 'gone', 'st', 'trip', 'yet', 'p'], ['flying', 'fabulous', 'seductive', 'sky', 'u', 'take', 'stress', 'away', 'travel'], ['thanks'], ['sfo', 'pdx', 'schedule', 'still', 'mia'], ['excited

In [45]:
help(nltk.pos_tag)

Help on function pos_tag in module nltk.tag:

pos_tag(tokens, tagset=None, lang='eng')
    Use NLTK's currently recommended part of speech tagger to
    tag the given list of tokens.
    
        >>> from nltk.tag import pos_tag
        >>> from nltk.tokenize import word_tokenize
        >>> pos_tag(word_tokenize("John's big idea isn't all that bad."))
        [('John', 'NNP'), ("'s", 'POS'), ('big', 'JJ'), ('idea', 'NN'), ('is', 'VBZ'),
        ("n't", 'RB'), ('all', 'PDT'), ('that', 'DT'), ('bad', 'JJ'), ('.', '.')]
        >>> pos_tag(word_tokenize("John's big idea isn't all that bad."), tagset='universal')
        [('John', 'NOUN'), ("'s", 'PRT'), ('big', 'ADJ'), ('idea', 'NOUN'), ('is', 'VERB'),
        ("n't", 'ADV'), ('all', 'DET'), ('that', 'DET'), ('bad', 'ADJ'), ('.', '.')]
    
    NB. Use `pos_tag_sents()` for efficient tagging of more than one sentence.
    
    :param tokens: Sequence of tokens to be tagged
    :type tokens: list(str)
    :param tagset: the tagset to be u

In [53]:
M=[]
for i in range(0,len(L)):
    pos_val = nltk.pos_tag(L[i])
    M+=[pos_val]
    pos=neg=obj=count=0
M    

[[('said', 'VBD')],
 [('plus', 'CC'),
  ('added', 'JJ'),
  ('commercial', 'JJ'),
  ('experience', 'NN'),
  ('tacky', 'NN')],
 [('today', 'NN'),
  ('must', 'MD'),
  ('mean', 'VB'),
  ('need', 'MD'),
  ('take', 'VB'),
  ('another', 'DT'),
  ('trip', 'NN')],
 [('really', 'RB'),
  ('aggressive', 'JJ'),
  ('blast', 'NN'),
  ('obnoxious', 'JJ'),
  ('entertainment', 'NN'),
  ('guest', 'JJS'),
  ('face', 'NN'),
  ('amp', 'NN'),
  ('little', 'JJ'),
  ('recourse', 'NN')],
 [('really', 'RB'), ('big', 'JJ'), ('bad', 'JJ'), ('thing', 'NN')],
 [('seriously', 'RB'),
  ('would', 'MD'),
  ('pay', 'VB'),
  ('flight', 'NN'),
  ('seat', 'NN'),
  ('playing', 'VBG'),
  ('really', 'RB'),
  ('bad', 'JJ'),
  ('thing', 'NN'),
  ('flying', 'VBG'),
  ('va', 'NN')],
 [('yes', 'RB'),
  ('nearly', 'RB'),
  ('every', 'DT'),
  ('time', 'NN'),
  ('fly', 'VBZ'),
  ('vx', 'JJ'),
  ('ear', 'NN'),
  ('worm', 'NN'),
  ('go', 'VB'),
  ('away', 'RB')],
 [('really', 'RB'),
  ('missed', 'JJ'),
  ('prime', 'JJ'),
  ('opportunity

In [47]:
# We can get more details about any POS tag using help funciton of NLTK as follows.
nltk.download('tagsets')
nltk.help.upenn_tagset("JJS$")

JJS: adjective, superlative
    calmest cheapest choicest classiest cleanest clearest closest commonest
    corniest costliest crassest creepiest crudest cutest darkest deadliest
    dearest deepest densest dinkiest ...


[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [54]:
for i in range(0,len(L)):
    nltk.download('universal_tagset')
    pos_val2 = nltk.pos_tag(L[i], tagset='universal')
    print(pos_val2)

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package un

[('said', 'VERB')]
[('plus', 'CONJ'), ('added', 'ADJ'), ('commercial', 'ADJ'), ('experience', 'NOUN'), ('tacky', 'NOUN')]
[('today', 'NOUN'), ('must', 'VERB'), ('mean', 'VERB'), ('need', 'VERB'), ('take', 'VERB'), ('another', 'DET'), ('trip', 'NOUN')]
[('really', 'ADV'), ('aggressive', 'ADJ'), ('blast', 'NOUN'), ('obnoxious', 'ADJ'), ('entertainment', 'NOUN'), ('guest', 'ADJ'), ('face', 'NOUN'), ('amp', 'NOUN'), ('little', 'ADJ'), ('recourse', 'NOUN')]
[('really', 'ADV'), ('big', 'ADJ'), ('bad', 'ADJ'), ('thing', 'NOUN')]
[('seriously', 'ADV'), ('would', 'VERB'), ('pay', 'VERB'), ('flight', 'NOUN'), ('seat', 'NOUN'), ('playing', 'VERB'), ('really', 'ADV'), ('bad', 'ADJ'), ('thing', 'NOUN'), ('flying', 'VERB'), ('va', 'NOUN')]
[('yes', 'ADV'), ('nearly', 'ADV'), ('every', 'DET'), ('time', 'NOUN'), ('fly', 'VERB'), ('vx', 'ADJ'), ('ear', 'NOUN'), ('worm', 'NOUN'), ('go', 'VERB'), ('away', 'ADV')]
[('really', 'ADV'), ('missed', 'ADJ'), ('prime', 'ADJ'), ('opportunity', 'NOUN'), ('men', 'N

[('interesting', 'VERB'), ('flying', 'VERB'), ('cancelled', 'ADJ'), ('flight', 'NOUN'), ('next', 'ADP'), ('four', 'NUM'), ('flight', 'NOUN'), ('planned', 'VERB'), ('neverflyvirginforbusiness', 'NOUN')]
[('disappointing', 'ADJ'), ('experience', 'NOUN'), ('shared', 'VERB'), ('every', 'DET'), ('business', 'NOUN'), ('traveler', 'NOUN'), ('meet', 'NOUN'), ('neverflyvirgin', 'NOUN')]
[('trouble', 'NOUN'), ('adding', 'VERB'), ('flight', 'NOUN'), ('wife', 'NOUN'), ('booked', 'VERB'), ('elevate', 'ADJ'), ('account', 'NOUN'), ('help', 'NOUN')]
[('bring', 'VERB'), ('reservation', 'NOUN'), ('online', 'NOUN'), ('using', 'VERB'), ('flight', 'NOUN'), ('booking', 'NOUN'), ('problem', 'NOUN'), ('code', 'NOUN')]
[('random', 'NOUN'), ('q', 'ADJ'), ('distribution', 'NOUN'), ('elevate', 'NOUN'), ('avatar', 'NOUN'), ('bet', 'VERB'), ('kitty', 'ADJ'), ('disproportionate', 'NOUN'), ('share', 'NOUN')]
[('lt', 'NOUN'), ('flying', 'VERB'), ('va', 'ADJ'), ('life', 'NOUN'), ('happens', 'VERB'), ('trying', 'VERB'),

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package un

[('understand', 'ADV'), ('need', 'VERB'), ('dm', 'VERB'), ('give', 'VERB'), ('answer', 'ADJ'), ('damaged', 'VERB'), ('luggage', 'NOUN'), ('policy', 'NOUN')]
[('mean', 'NOUN'), ('policy', 'NOUN'), ('destroyed', 'VERB'), ('luggage', 'NOUN')]
[('special', 'ADJ'), ('assistance', 'NOUN'), ('travel', 'NOUN'), ('alone', 'ADV'), ('w', 'ADJ'), ('kid', 'NOUN'), ('infant', 'NOUN'), ('priority', 'NOUN'), ('boarding', 'NOUN')]
[('thank', 'NOUN'), ('checking', 'VERB'), ('ticket', 'NOUN'), ('purchased', 'VERB'), ('customer', 'NOUN'), ('happy', 'ADJ')]
[('website', 'ADJ'), ('ever', 'ADV'), ('coming', 'VERB'), ('back', 'ADV'), ('online', 'NOUN')]
[('flight', 'NOUN'), ('love', 'NOUN'), ('field', 'NOUN'), ('sfo', 'NOUN'), ('definitely', 'ADV'), ('cancelled', 'VERB'), ('flightled', 'VERB'), ('monday', 'ADV'), ('february', 'ADJ')]
[('flight', 'NOUN'), ('lga', 'NOUN'), ('dfw', 'NOUN'), ('time', 'NOUN')]
[('driver', 'NOUN'), ('license', 'NOUN'), ('expired', 'VERB'), ('little', 'ADJ'), ('month', 'NOUN'), ('fl

...

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package un

[('appreciate', 'NOUN'), ('effort', 'NOUN'), ('getting', 'VERB'), ('home', 'NOUN')]
[('correct', 'ADJ'), ('spent', 'VERB'), ('hour', 'NOUN'), ('trying', 'VERB'), ('book', 'NOUN'), ('online', 'ADJ'), ('receive', 'ADJ'), ('error', 'NOUN'), ('clicking', 'VERB'), ('final', 'ADJ'), ('purchase', 'NOUN'), ('button', 'NOUN')]
[('today', 'NOUN'), ('finest', 'ADJ'), ('could', 'VERB'), ('prevented', 'VERB'), ('one', 'NUM'), ('gate', 'NOUN'), ('agent', 'NOUN'), ('advising', 'VERB'), ('passenger', 'NOUN')]
[('real', 'ADJ'), ('deal', 'NOUN'), ('delay', 'NOUN'), ('flight', 'NOUN'), ('ua', 'NOUN'), ('denver', 'NOUN'), ('vega', 'NOUN')]
[('ua', 'ADJ'), ('madrid', 'NOUN'), ('arrived', 'VERB'), ('agent', 'ADJ'), ('mass', 'NOUN'), ('confusion', 'NOUN'), ('sent', 'VERB'), ('fro', 'NOUN'), ('ua', 'ADJ'), ('employee', 'NOUN'), ('maryjo', 'NOUN'), ('rude', 'VERB'), ('unhelpful', 'ADJ')]
[('system', 'NOUN'), ('failure', 'NOUN'), ('bad', 'ADJ'), ('trend', 'NOUN'), ('software', 'NOUN'), ('project', 'NOUN'), ('li

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package un

[('c', 'NOUN'), ('think', 'VERB'), ('mile', 'ADP'), ('repair', 'NOUN'), ('damage', 'NOUN'), ('done', 'VERB'), ('interesting', 'ADJ'), ('travel', 'NOUN'), ('exp', 'NOUN'), ('worst', 'ADJ'), ('flight', 'NOUN'), ('ever', 'ADV')]
[('flight', 'NOUN'), ('dfw', 'NOUN'), ('tuesday', 'NOUN'), ('far', 'ADV'), ('advance', 'ADV'), ('give', 'ADJ'), ('notice', 'NOUN'), ('cancelled', 'VERB'), ('flightled', 'VERB')]
[('signed', 'VERB'), ('notification', 'NOUN'), ('first', 'ADV'), ('trip', 'NOUN'), ('updated', 'ADJ'), ('sure', 'NOUN'), ('happened', 'VERB')]
[('thank', 'NOUN'), ('took', 'VERB'), ('care', 'NOUN'), ('everything', 'NOUN'), ('made', 'VERB'), ('right', 'ADJ'), ('experience', 'NOUN'), ('used', 'VERB'), ('opened', 'VERB'), ('app', 'ADJ'), ('find', 'VERB'), ('flight', 'NOUN'), ('changed', 'NOUN')]
[('line', 'NOUN'), ('response', 'NOUN'), ('waiting', 'VERB'), ('tomorrow', 'NOUN'), ('morning', 'NOUN'), ('flight', 'NOUN'), ('overbooked', 'VERB')]
[('took', 'VERB'), ('exact', 'NOUN'), ('aircraft', 

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package un

In [50]:
# Convert between the PennTreebank tags to simple Wordnet tags
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [51]:
# Returns list of pos-neg and objective score. But returns empty list if not present in senti wordnet.
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    #Lemmatization
    #lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    #if not lemma:
     #   return []

    #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
    #Synset instances are the groupings of synonymous words that express the same concept. 
    #Some of the words have only one Synset and some have several.
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

In [60]:
S=[]
for i in range(0,len(M)):
    senti_val = [get_sentiment(x,y) for (x,y) in M[i]]
    S+=senti_val
S    

[[],
 [],
 [],
 ['commercial.a.01', 0.0, 0.0, 1.0],
 ['experience.n.01', 0.0, 0.0, 1.0],
 [],
 ['today.n.01', 0.125, 0.0, 0.875],
 [],
 [],
 [],
 [],
 [],
 ['trip.n.01', 0.0, 0.0, 1.0],
 ['truly.r.01', 0.625, 0.0, 0.375],
 ['aggressive.a.01', 0.5, 0.0, 0.5],
 ['blast.n.01', 0.0, 0.125, 0.875],
 ['objectionable.s.01', 0.0, 0.625, 0.375],
 ['entertainment.n.01', 0.0, 0.0, 1.0],
 [],
 ['face.n.01', 0.0, 0.0, 1.0],
 ['ampere.n.02', 0.0, 0.0, 1.0],
 ['small.a.01', 0.0, 0.375, 0.625],
 ['recourse.n.01', 0.0, 0.0, 1.0],
 ['truly.r.01', 0.625, 0.0, 0.375],
 ['large.a.01', 0.25, 0.125, 0.625],
 ['bad.a.01', 0.0, 0.625, 0.375],
 ['thing.n.01', 0.0, 0.0, 1.0],
 ['seriously.r.01', 0.25, 0.0, 0.75],
 [],
 [],
 ['flight.n.01', 0.0, 0.0, 1.0],
 ['seat.n.01', 0.0, 0.0, 1.0],
 [],
 ['truly.r.01', 0.625, 0.0, 0.375],
 ['bad.a.01', 0.0, 0.625, 0.375],
 ['thing.n.01', 0.0, 0.0, 1.0],
 [],
 ['virginia.n.01', 0.0, 0.0, 1.0],
 [],
 ['about.r.07', 0.0, 0.0, 1.0],
 [],
 ['time.n.01', 0.0, 0.0, 1.0],
 [],
 [],


In [64]:
for i in range(len(S)):
  try:
    pos = pos + S[i][1]
    neg = neg + S[i][2]
    obj = obj + S[i][3]
  
  except:
    continue

In [65]:
print("Positive weight : {0} ".format(pos))
print("Negative weight : {0} ".format(neg))
print("Sentiment of the statement is {0} ".format(pos - neg))

Positive weight : 347.125 
Negative weight : 310.5 
Sentiment of the statement is 36.625 
